In [77]:
import numpy as np
import pandas as pd
import itertools

In [78]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [79]:
df = pd.read_csv("news.csv")
df.shape
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [80]:
labels = df.label
labels.head(10)

0    FAKE
1    FAKE
2    REAL
3    FAKE
4    REAL
5    FAKE
6    FAKE
7    REAL
8    REAL
9    REAL
Name: label, dtype: object

In [81]:
x_train,x_test,y_train,y_test=train_test_split(df['text'],labels,test_size=0.2,random_state=7)

In [82]:
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)

In [83]:
tfidf_train= tfidf_vectorizer.fit_transform(x_train)
tfidf_test = tfidf_vectorizer.transform(x_test)

In [84]:
#DataFlair - Initialize a PassiveAggressiveClassifier
pac=PassiveAggressiveClassifier(max_iter=500)
pac.fit(tfidf_train,y_train)
#DataFlair - Predict on the test set and calculate accuracy
y_pred=pac.predict(tfidf_test)
score=accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 92.58%


In [85]:
#DataFlair - Build confusion matrix
confusion_matrix(y_test,y_pred, labels=['FAKE','REAL'])

array([[589,  49],
       [ 45, 584]])

In [86]:
def predict_news(news_text):
    # Transform the input text using the already fitted tfidf_vectorizer
    vectorized_text = tfidf_vectorizer.transform([news_text])
    
    # Get decision score (PassiveAggressiveClassifier does not have predict_proba)
    score = pac.decision_function(vectorized_text)[0]
    
    # Predict the label
    prediction = pac.predict(vectorized_text)[0]
    
    # Convert raw score to a confidence value between 0 and 1
    # (Higher absolute value = higher confidence)
    confidence = 1 / (1 + np.exp(-abs(score)))  # sigmoid
    
    return prediction, round(confidence * 100, 2)


In [87]:
predict_news("Donald trump died yesterday")

('FAKE', 73.27)

In [88]:
predict_news("Alaska Meet Live Updates: After Putin Meeting, Ukraine's Zelensky To Meet Trump In Washington On Monday")

('REAL', 75.11)